In [114]:
import pandas as pd 

df = pd.read_csv('HR.csv')

df.head(5)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   left                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   sales                  14999 non-null  object 
 9   salary                 14999 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


## 1 - we dont have a null data
 

## 2 - we need to label encoding the sales and salary cloumns for OHE preprocessing

In [116]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
cat_cols = df.select_dtypes(include=['object']).columns
for col in cat_cols:
    df[col] = le.fit_transform(df[col])
df[cat_cols] = df[cat_cols].apply(lambda col: le.fit_transform(col))
df.head(5)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,7,1
1,0.80,0.86,5,262,6,0,1,0,7,2
2,0.11,0.88,7,272,4,0,1,0,7,2
3,0.72,0.87,5,223,5,0,1,0,7,1
4,0.37,0.52,2,159,3,0,1,0,7,1


## Separate data for target and train

In [117]:
import numpy as np 
df = df.iloc[ : , : ].values

x = np.delete(df, [-4],axis=1)

y = df[ : , -4 ]

print(x[0])
print(y[0])

[  0.38   0.53   2.   157.     3.     0.     0.     7.     1.  ]
1.0


In [118]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([(('sales','salary'),OneHotEncoder(),  [ -2, -1] )],remainder='passthrough')
x = ct.fit_transform(x)

x[0]

array([  0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,
         0.  ,   0.  ,   0.  ,   1.  ,   0.  ,   0.38,   0.53,   2.  ,
       157.  ,   3.  ,   0.  ,   0.  ])

## 3 - Standard Scaler

In [119]:
# Standard Scaler

from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
x = ss.fit_transform(x)

x[ 0 ]



array([-0.29848587, -0.2353205 , -0.23214788, -0.22764728, -0.20939051,
       -0.24632222, -0.25295305,  1.61955144, -0.41779149, -0.4706553 ,
       -0.29980859,  1.02477511, -0.8681323 , -0.93649469, -1.08727529,
       -1.46286291, -0.88203988, -0.34123516, -0.41116529, -0.14741182])

## Algorithm XGB

In [120]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=123)
param_grid = {
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300]
}
xgb = XGBRegressor()
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
# Calculate the R2 score
r2 = r2_score(y_test, y_pred)
print("R2 Score: ", r2)

R2 Score:  0.9426829804670901


## عملکردی که ما اینجا به دست آوردیم %94 است که یعنی اگه ما دیتا جدیدی به دست بیاریم و به الگوریتم بدیم که پیش بینی کنه که آیا این کارمند استفاء میدهد یا خیر را با احتمال 94 درصد درست پیش بینی میکند و احتمال خطای 6 درصد دارد که برای این مدل درصد خطای خوبی است